# **Data Collection- Collect Data from Kaggle**

## Objectives

* Fetch data from Kaggle and prepare it for further processes.

## Inputs

*   Kaggle JSON file - the authentication token. 

## Outputs

* Generate Dataset: inputs/datasets/cherry_leaves

## Additional Comments | Insights | Conclusions


* No additional comments.



---


# Import packages

In [1]:
%pip install -r /workspace/project5/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy
import os

---


# Change working directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/project5/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [4]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'/workspace/project5'

# Install Kaggle

Install Kaggle package

In [11]:
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [7]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

In [16]:
KaggleDatasetPath = "datasets/codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_leaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

403 - Forbidden


In [17]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry_leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry_leaves.zip')

FileNotFoundError: [Errno 2] No such file or directory: 'inputs/cherry_leaves_dataset/malaria-cell-classification.zip'

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [9]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)


IndentationError: expected an indented block (1114530593.py, line 5)